# Healthcare No Show CSV to SQL

Scripts for building SQL tables and store (cleaned) data.

In [1]:
import os
from dotenv import load_dotenv
import mysql.connector
import sqlalchemy
import pandas as pd

load_dotenv()
pd.set_option("display.expand_frame_repr", False)

Create table

In [2]:
mydb = mysql.connector.connect(
    host=os.getenv("HOST"),
    user=os.getenv("USER"),
    password=os.getenv("PASSWORD"),
    database=os.getenv("DATABASE"),
)
mycursor = mydb.cursor()

In [3]:
create_table_query = (
    "CREATE TABLE healthcare ("
    "appointment_id MEDIUMINT UNSIGNED PRIMARY KEY, "
    "patient_id BIGINT UNSIGNED, "
    'gender ENUM("F", "M"), '
    "scheduled_day DATETIME, "
    "appointment_day DATETIME, "
    "age TINYINT UNSIGNED, "
    "neighbourhood VARCHAR(255), "
    "scholarship BIT(1), "
    "hypertension BIT(1), "
    "diabetes BIT(1), "
    "alcoholism BIT(1), "
    "handicap TINYINT UNSIGNED, "
    "sms_received BIT(1), "
    "no_show BIT(1) "
    ");"
)

In [4]:
mycursor.execute(create_table_query)

In [5]:
mycursor.close()

True

Load data from CSV to SQL

In [ ]:
file_path = "../../data/healthcare_no_show/KaggleV2-May-2016.csv"
pd_data = pd.read_csv(file_path)
print(pd_data)

           PatientId  AppointmentID Gender          ScheduledDay        AppointmentDay  Age      Neighbourhood  Scholarship  Hipertension  Diabetes  Alcoholism  Handcap  SMS_received No-show
0       2.987250e+13        5642903      F  2016-04-29T18:38:08Z  2016-04-29T00:00:00Z   62    JARDIM DA PENHA            0             1         0           0        0             0      No
1       5.589978e+14        5642503      M  2016-04-29T16:08:27Z  2016-04-29T00:00:00Z   56    JARDIM DA PENHA            0             0         0           0        0             0      No
2       4.262962e+12        5642549      F  2016-04-29T16:19:04Z  2016-04-29T00:00:00Z   62      MATA DA PRAIA            0             0         0           0        0             0      No
3       8.679512e+11        5642828      F  2016-04-29T17:29:31Z  2016-04-29T00:00:00Z    8  PONTAL DE CAMBURI            0             0         0           0        0             0      No
4       8.841186e+12        5642494      F  2

In [7]:
# Rename field names
pd_data.rename(
    columns={
        "PatientId": "patient_id",
        "AppointmentID": "appointment_id",
        "Gender": "gender",
        "ScheduledDay": "scheduled_day",
        "AppointmentDay": "appointment_day",
        "Age": "age",
        "Neighbourhood": "neighbourhood",
        "Scholarship": "scholarship",
        "Hipertension": "hypertension",
        "Diabetes": "diabetes",
        "Alcoholism": "alcoholism",
        "Handcap": "handicap",
        "SMS_received": "sms_received",
        "No-show": "no_show",
    },
    inplace=True,
)
print(pd_data)

          patient_id  appointment_id gender         scheduled_day       appointment_day  age      neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received no_show
0       2.987250e+13         5642903      F  2016-04-29T18:38:08Z  2016-04-29T00:00:00Z   62    JARDIM DA PENHA            0             1         0           0         0             0      No
1       5.589978e+14         5642503      M  2016-04-29T16:08:27Z  2016-04-29T00:00:00Z   56    JARDIM DA PENHA            0             0         0           0         0             0      No
2       4.262962e+12         5642549      F  2016-04-29T16:19:04Z  2016-04-29T00:00:00Z   62      MATA DA PRAIA            0             0         0           0         0             0      No
3       8.679512e+11         5642828      F  2016-04-29T17:29:31Z  2016-04-29T00:00:00Z    8  PONTAL DE CAMBURI            0             0         0           0         0             0      No
4       8.841186e+12         564249

In [8]:
pd_data["patient_id"] = pd_data["patient_id"].astype("int64")
pd_data["scheduled_day"] = pd.to_datetime(pd_data["scheduled_day"])
pd_data["appointment_day"] = pd.to_datetime(pd_data["appointment_day"])
pd_data["age"] = pd_data["age"].apply(lambda x: x if x >= 0 else None)
pd_data["no_show"] = pd_data["no_show"].apply(lambda x: 1 if x == "Yes" else 0)
print(pd_data)

             patient_id  appointment_id gender             scheduled_day           appointment_day   age      neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show
0        29872499824296         5642903      F 2016-04-29 18:38:08+00:00 2016-04-29 00:00:00+00:00  62.0    JARDIM DA PENHA            0             1         0           0         0             0        0
1       558997776694438         5642503      M 2016-04-29 16:08:27+00:00 2016-04-29 00:00:00+00:00  56.0    JARDIM DA PENHA            0             0         0           0         0             0        0
2         4262962299951         5642549      F 2016-04-29 16:19:04+00:00 2016-04-29 00:00:00+00:00  62.0      MATA DA PRAIA            0             0         0           0         0             0        0
3          867951213174         5642828      F 2016-04-29 17:29:31+00:00 2016-04-29 00:00:00+00:00   8.0  PONTAL DE CAMBURI            0             0         0           0    

In [9]:
engine = sqlalchemy.create_engine(
    f"mysql+mysqlconnector://{os.getenv('USER')}:{os.getenv('PASSWORD')}@{os.getenv('HOST')}:3306/{os.getenv('DATABASE')}"
)

In [10]:
pd_data.to_sql(name="healthcare", con=engine, if_exists="append", index=False)

110527

In [11]:
with engine.connect() as conn:
    result = conn.execute(
        sqlalchemy.text("SELECT * FROM healthcare LIMIT 10;")
    ).fetchall()

print(result)

[(5030230, 832256398961987, 'F', datetime.datetime(2015, 11, 10, 7, 13, 56), datetime.datetime(2016, 5, 4, 0, 0), 51, 'RESISTÊNCIA', 0, 0, 0, 0, 0, 1, 0), (5122866, 91637474953513, 'M', datetime.datetime(2015, 12, 3, 8, 17, 28), datetime.datetime(2016, 5, 2, 0, 0), 34, 'VILA RUBIM', 0, 1, 0, 0, 0, 1, 1), (5134197, 1216586867796, 'F', datetime.datetime(2015, 12, 7, 10, 40, 59), datetime.datetime(2016, 6, 3, 0, 0), 27, 'SÃO CRISTÓVÃO', 1, 0, 0, 0, 0, 1, 1), (5134220, 31899595421534, 'F', datetime.datetime(2015, 12, 7, 10, 42, 42), datetime.datetime(2016, 6, 3, 0, 0), 48, 'MARUÍPE', 0, 1, 1, 0, 0, 1, 0), (5134223, 9582232334148, 'F', datetime.datetime(2015, 12, 7, 10, 43, 1), datetime.datetime(2016, 6, 3, 0, 0), 80, 'SÃO CRISTÓVÃO', 0, 1, 1, 0, 0, 1, 0), (5134224, 3516253533716, 'F', datetime.datetime(2015, 12, 7, 10, 43, 17), datetime.datetime(2016, 6, 3, 0, 0), 74, 'SÃO CRISTÓVÃO', 0, 1, 1, 0, 0, 1, 0), (5134227, 454287126844, 'M', datetime.datetime(2015, 12, 7, 10, 43, 34), datetime.da

In [12]:
engine.dispose()